In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
import seaborn as sns
from numpy import array

In [ ]:
data = pd.read_csv(r"C:\Users\ashis\Desktop\PS1\final_complete_hourly_data_2016_2019.csv")
data = pd.DataFrame(data)
data['date'] = pd.to_datetime(data['date'])

X = data[['tempr', 'ap', 'ws', 'rh']]
y = data['blizzard']



In [ ]:
train = int(len(X)*0.8)
X_train = X[:train]
y_train = y[:train]
X_test = X[train:]
y_test = y[train:]

y_test = array(y_test)

ap_test = data['ap'][train:]
ap_test = array(ap_test)
speed_test = data['ws'][train:]
speed_test = array(speed_test)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
model.fit(X_train_scaled, y_train, epochs=200, batch_size=32, verbose=1)

In [ ]:
y_pred_prob = model.predict(X_test_scaled)
y_pred = (y_pred_prob > 0.5).astype(int)

y_test = array(y_test)

y_test

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as mpatches

res=[]
outk = []
for i in range(len(y_pred)):
    res.append(y_pred[i])
res = array(res)
res = (res-min(res))/(max(res)-min(res))
tn = fn = tp = fp = pos = neg = 0
for i in range(len(res)):
    if res[i] > 0.5:
        res[i] = 1
    else:
        res[i] = 0

for i in range(len(res)):
    if y_test[i] == 0:
        neg += 1
        if res[i] == 0:
            outk.append(0)
            tn += 1
        else:
            outk.append(2)
            fp += 1
    else:
        pos += 1
        if res[i] == 1:
            outk.append(1)
            tp += 1
        else:
            outk.append(3)
            fn += 1
            
print("positive: ", pos, " negative: ", neg, " tp: ", tp, " tn: ", tn, " fn: ", fn, " fp: ", fp)
print(2*tp/(2*tp +fp+fn))



outk = outk[-len(y_test):]


plt.figure(figsize=(17,6))
ax1 = plt.gca()  # Get current axis
ax2 = ax1.twinx()  # Create a twin axis

# Plot wind speed on the left y-axis
ax1.plot(data['date'][-8904:], data['ws'][-8904:], linewidth=0.5, color='blue')
ax1.set_ylabel('Wind speed (m/s)')

# Plot adjusted air pressure on the right y-axis
ax2.plot(data['date'][-8904:], (data['ap'][-8904:]), linewidth=0.5, color='orange')
ax2.set_ylabel('Air Pressure(Bar)')

# Set the scale of the right y-axis
ax2.set_ylim(950, 1000)
plt.xlabel("Date", fontweight = 'bold' )

ax1.set_xlabel('Date')
plt.title("Classification of Blizzard using ANN\n Duration: 2020-01-29 to 2021-02-04\n Training Duration: 2016-01-01 to 2020-01-29", fontweight = 'bold', fontsize = 10)
for i in range(len(speed_test)):
    if outk[i] == 1:
        ax1.scatter(data['date'][35750+i], speed_test[i], s=20 ,c='green')
    elif outk[i] == 2:
        ax1.scatter(data['date'][35750+i], speed_test[i],s=20 ,c='yellow')
    elif outk[i] == 3:
        ax1.scatter(data['date'][35750+i], speed_test[i], s=20,c='red')


pop_a = mpatches.Patch(color='red', label='Missed')
pop_b = mpatches.Patch(color='yellow', label='Incorrect')
pop_c = mpatches.Patch(color='green', label='Correct')
pop_d = mpatches.Patch(color='blue', label='Wind Speed')
pop_e = mpatches.Patch(color = 'orange', label = 'Air Pressure')
plt.legend(handles=[pop_a,pop_b, pop_c, pop_d,pop_e])
plt.savefig("Blizzard_ANN.png", format='png', dpi=800)

plt.show()


In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

In [ ]:
f1 = f1_score(y_test, y_pred)
print("F1 Score:", f1)